In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import random
import pandas as pd
import pickle
# import xml.etree.ElementTree as ET
import scipy.io as scio
from utils.boxes import bbox_overlaps
from PIL import Image
import json
from pprint import pprint
import sng_parser
import ipdb

import torch
# from pytorch_pretrained_bert.tokenization import BertTokenizer
# from pytorch_pretrained_bert.modeling import BertModel
from skipthoughts import UniSkip

In [2]:
# # create Selective Search Segmentation Object using default parameters
# cv2.setUseOptimized(True);
# cv2.setNumThreads(4);
# ss = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()
# def ss_boxes(img, mode='f', keep_boxes=100):
#     # read image
#     # set input image on which we will run segmentation
#     ss.setBaseImage(img)
 
#     # Switch to fast but low recall Selective Search method
#     if (mode == 'f'):
#         ss.switchToSelectiveSearchFast()
#     # Switch to high recall but slow Selective Search method
#     elif (mode == 'q'):
#         ss.switchToSelectiveSearchQuality()
 
#     # run selective search segmentation on input image
#     rects = ss.process()[:keep_boxes, :]
# #     print('Total Number of Region Proposals: {}'.format(len(rects)))
#     rects[:, 2] += rects[:, 0]
#     rects[:, 3] += rects[:, 1]
#     return rects

In [3]:
sentence_root = '/p300/flickr30k_images/flickr30k_anno/sent_anno.json'
bbox_root = '/p300/flickr30k_images/flickr30k_anno/box_anno.json'
im_root = '/p300/flickr30k_images/flickr30k_images/'

In [4]:
sent_anno = json.load(open(sentence_root, 'r'))
bbox_anno = json.load(open(bbox_root, 'r'))

In [5]:
im_ids = list(sent_anno.keys())

## Denoise

In [6]:
# errors = np.array([129602216, 5,
# 133010954, 5,
# 153299765, 2,
# 158388874, 5,
# 200767661, 5,
# 3367384342, 2,
# 4671832284, 1,
# 7638876050, 5,
# 3652094744, 1,
# 7017792809, 5,
# 7052778829, 5,
# 7232861768, 5,
# 4901396689, 5,
# 6154676236, 5,
# 6442477951, 5,
# 4351142771, 5,
# 5096654568, 5,
# 100759042, 5,
# 5566135246, 5,
# 2190899457, 5,
# 180753784, 3])
# errors = errors.reshape(len(errors)//2,2)
# for (im_id, sent_id) in errors:
#     del sent_anno[str(im_id)][sent_id-1]
# with open(sentence_root, 'w') as f:
#     json.dump(sent_anno, f)

In [7]:
# sent_anno_new = {}

# for im_id, all_sents in sent_anno.items():
#     boxes = bbox_anno[im_id]
#     all_phrase_ids = []
#     for k, v in boxes['boxes'].items():
#         all_phrase_ids.append(k)
    
#     all_sents_new = []
#     for sent_id, sent in enumerate(all_sents):
#         phrases = sent['phrases']
#         phrase_ids = []
#         for phr in phrases:
#             if phr['phrase_id'] not in all_phrase_ids:
#                 continue
#             phrase_ids.append(phr['phrase_id'])
        
#         if len(phrase_ids) > 0:
#             all_sents_new.append(sent)
#     sent_anno_new[im_id] = all_sents_new
            
# with open(sentence_root, 'w') as f:
#     json.dump(sent_anno_new, f)

In [16]:
# train_f = '/p300/flickr30k_images/split/train.txt'
# test_f = '/p300/flickr30k_images/split/test.txt'
# val_f = '/p300/flickr30k_images/split/val.txt'

# cur_file = val_f
# with open(cur_file) as ff:
#     cur = ff.readlines()
    
# with open(cur_file+'.tmp', 'w') as fff:
#     for i in cur:
#         if i.strip() in im_ids:
#             fff.write(i)
            
# with open(cur_file+'.tmp', 'w') as fff:
#     for i, value in enumerate(cur):
#         im_id = value.strip()
#         sent_anno_num = len(sent_anno[im_id])
#         for j in range(sent_anno_num):
#             fff.write(im_id+'\t'+str(j)+'\n')
            
            
# sent_anno_new = {}
# for im_id, sent in sent_anno.items():
#     if len(sent) > 0:
#         sent_anno_new[im_id] = sent
# with open(sentence_root, 'w') as f:
#     json.dump(sent_anno_new, f)

In [40]:
with open('/p300/flickr30k_images/split/val_clean.txt') as ff:
    res = ff.readlines()
len(res)

1000

### upper bound

In [24]:
vg_pred_root = '/p300/flickr30k_images/flickr30k_anno/precompute_proposals_nms_1e4.json'
vg_pred = json.load(open(vg_pred_root))

In [25]:
len(vg_pred)

31783

In [29]:
np.array(vg_pred['100652400']).shape

(100, 5)

In [30]:
all_gt_num = 0
all_recall_num = 0
for im_id in im_ids:
    # get sentence and sentence anno
    st_anno = sent_anno[im_id]
    # get annotation
    gt_anno = bbox_anno[im_id]
    
    # get gt boxes
    gt_boxes = []
    gt_box_ids = []
    for pid, v in gt_anno['boxes'].items():
        gt_box_ids.append(pid)
        if len(v) == 1:
            gt_boxes.append(v[0])
        else:
            # when a phrase respond to multiple regions, we take the union of them as paper given
            v = np.array(v)
            box = [v[:,0].min(), v[:,1].min(), v[:,2].max(), v[:,3].max()]
            gt_boxes.append(box)
    gt_boxes = np.array(gt_boxes)
    
    gt_num = gt_boxes.shape[0]
    all_gt_num += gt_num
    
    ## given ss boxes from https://github.com/kanchen-usc/QRC-Net/blob/master/feature/README.md
    #     rects = scio.loadmat(ss_boxes_root + im_id + '.mat')['cur_bbxes']
    
    ## ss boxes we compute
#     im = cv2.imread(im_root + im_id + '.jpg')
#     rects = ss_boxes(im, mode='f', keep_boxes=100)

    ## boxes from pretrained resnet101-fpn on vg dataset
#     rects = vg_pred[im_id].bbox.numpy()
#     scores = vg_pred[im_id].extra_fields['scores']
    rects = np.array(vg_pred[im_id])[:,:4]
    
    overlaps = bbox_overlaps(gt_boxes.astype(np.float32), rects.astype(np.float32))
    recall_num = np.sum(overlaps.max(1)>0.5)
    all_recall_num += recall_num

In [31]:
print(all_gt_num, all_recall_num, all_recall_num/all_gt_num)

193020 174496 0.90403067039685


### NLP

In [10]:
parser = sng_parser.Parser('spacy', model='en')

In [11]:
im_id = im_ids[1000]
st_anno = sent_anno[im_id]
sentence = st_anno[0]['sentence']
print(sentence)
phrases = st_anno[0]['phrases']
pprint(phrases)
graph = parser.parse(sentence)
sng_parser.tprint(graph)

A woman in a red long-sleeved shirt and blue jeans stands at the lectern speaking at or about GLAM U WIKI .
[{'first_word_index': 0,
  'phrase': 'A woman',
  'phrase_id': '142915',
  'phrase_type': ['people']},
 {'first_word_index': 3,
  'phrase': 'a red long-sleeved shirt',
  'phrase_id': '142918',
  'phrase_type': ['clothing']},
 {'first_word_index': 8,
  'phrase': 'blue jeans',
  'phrase_id': '142920',
  'phrase_type': ['clothing']},
 {'first_word_index': 12,
  'phrase': 'the lectern',
  'phrase_id': '142919',
  'phrase_type': ['other']},
 {'first_word_index': 18,
  'phrase': 'GLAM U WIKI',
  'phrase_id': '142916',
  'phrase_type': ['other']}]
Entities:
+-------------+-----------------------------------------+---------------+
| Head        | Span                                    | Modifiers     |
|-------------+-----------------------------------------+---------------|
| woman       | a woman                                 | a             |
| jeans       | a red long-sleeved shir

In [43]:
all_gt_num = 0
all_recall_num = 0
error_types = {}
correct_types = {}
for im_id in im_ids:
    st_anno = sent_anno[im_id]
    phrase_type_dict = {}
    for sent in st_anno:
        phrases = sent['phrases']
        for phrase in phrases:
            phrase_type_dict[phrase['phrase_id']] = phrase['phrase_type'][0]

    gt_anno = bbox_anno[im_id]

    # get gt boxes
    gt_boxes = []
    gt_box_type = []
    for pid, v in gt_anno['boxes'].items():
        gt_box_type.append(phrase_type_dict.get(pid))
        if len(v) == 1:
            gt_boxes.append(v[0])
        else:
            # when a phrase respond to multiple regions, we take the union of them as paper given
            v = np.array(v)
            box = [v[:,0].min(), v[:,1].min(), v[:,2].max(), v[:,3].max()]
            gt_boxes.append(box)
    gt_boxes = np.array(gt_boxes)

    gt_num = gt_boxes.shape[0]
    all_gt_num += gt_num

    rects = vg_pred[im_id+'.jpg'].bbox.numpy()

    overlaps = bbox_overlaps(gt_boxes.astype(np.float32), rects[:100, :].astype(np.float32))
    pos_ind = np.where(overlaps.max(1)>=0.5)[0]
    
    recall_num = len(pos_ind)
    all_recall_num += recall_num
    
    
    for psid in pos_ind:
        tmp_num = correct_types.setdefault(gt_box_type[psid], 0)
        tmp_num += 1
        correct_types[gt_box_type[psid]] = tmp_num
    
    
    neg_ind = np.where(overlaps.max(1)<0.5)[0]
    for ngid in neg_ind:
        tmp_num = error_types.setdefault(gt_box_type[ngid], 0)
        tmp_num += 1
        error_types[gt_box_type[ngid]] = tmp_num
print(all_gt_num, all_recall_num, all_recall_num/all_gt_num)
print(correct_types)
print(error_types)

193020 158352 0.8203916692570719
{'people': 51986, 'other': 36446, 'bodyparts': 6874, 'scene': 19335, 'clothing': 34635, 'vehicles': 3750, 'instruments': 1062, 'animals': 4251, None: 13}
{'bodyparts': 5036, 'clothing': 6615, 'scene': 3023, 'animals': 444, 'people': 3995, 'other': 14283, 'vehicles': 585, 'instruments': 683, None: 4}


In [13]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

In [49]:
dir_st = 'data/skip-thoughts'

In [69]:
# im_id = im_ids[0]
vocab = set()
for im_id in im_ids:
    st_anno = sent_anno[im_id]
    for sent in st_anno:
        seq = sent['sentence'].lower()
        tokenized_seq = seq.split(' ')

        vocab.update(tokenized_seq)
vocab = list(vocab)
print(len(vocab))

20214


In [71]:
vocab_to_id = {v:i+1 for i,v in enumerate(vocab)}

In [74]:
uniskip = UniSkip(dir_st, vocab)

In [75]:
error_count = 0

im_id = im_ids[0]
st_anno = sent_anno[im_id]
phrase_type_dict = {}
for sent in st_anno:
    seq = sent['sentence'].lower()
    tokenized_seq = seq.split(' ')

    phrases = sent['phrases']
    
    inputs = torch.zeros((len(phrases), len(tokenized_seq))).long()
    lengths = []
    
    for pid, phr in enumerate(phrases):
        tokenized_phr = phr['phrase'].lower().split(' ')
        phr_len = len(tokenized_phr)
        lengths.append(phr_len)
        
        start_ind = phr['first_word_index']
        end_ind = start_ind + phr_len
        
        phr_idx = []
        for w in tokenized_phr:
            phr_idx.append(vocab_to_id[w])
        
        inputs[pid][:phr_len] = torch.Tensor(phr_idx).long()
        
        assert tokenized_seq[start_ind:end_ind] == tokenized_phr


In [77]:
print(inputs, lengths)

tensor([[ 9278, 18539,     0,     0,     0,     0,     0,     0,     0,     0],
        [ 9278, 15584,     0,     0,     0,     0,     0,     0,     0,     0],
        [15311,  3098,     0,     0,     0,     0,     0,     0,     0,     0]]) [2, 2, 2]


In [78]:
embeds = uniskip(inputs, lengths=lengths)
print(embeds.shape)

torch.Size([3, 2400])


In [1]:
# error_count = 0
# for im_id in im_ids:
# #     im_id = im_ids[0]
#     st_anno = sent_anno[im_id]
#     phrase_type_dict = {}
#     for sent in st_anno:
#         seq = sent['sentence'].lower()
#         seq = "[CLS] " + seq + " [SEP]"
#         tokenized_seq = seq.split(' ')
#         try:
#             indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_seq)
#         except:
#             ipdb.set_trace()
# #         tokens_tensor = torch.tensor(indexed_tokens)

#         phrases = sent['phrases']
#         for phr in phrases:
#             tokenized_phr = phr['phrase'].lower().split(' ')
#             start_ind = phr['first_word_index'] + 1
#             end_ind = start_ind + len(tokenized_phr)
#             if tokenized_seq[start_ind:end_ind] != tokenized_phr:
#     #             print(tokenized_seq[start_ind:end_ind])
#     #             print(tokenized_phr)
#                 error_count += 1
            
                
# print(error_count)
# #         assert tokenized_seq[start_ind:end_ind] == tokenized_phr

In [38]:
tokenized_phr

['a', 'harness']

In [32]:
seq.split(' ')

['[CLS]',
 'A',
 'woman',
 'dressed',
 'in',
 'black',
 'is',
 'walking',
 'her',
 'son',
 'with',
 'a',
 'harness',
 '.',
 '[SEP]']

In [14]:
st_anno[0]

{'sentence': 'A man walked across a street holding a basket on top of his head filled with fruit and vegetables with one hand , while holding bags of grapes in his right hand .',
 'phrases': [{'first_word_index': 0,
   'phrase': 'A man',
   'phrase_id': '84695',
   'phrase_type': ['people']},
  {'first_word_index': 4,
   'phrase': 'a street',
   'phrase_id': '84710',
   'phrase_type': ['scene']},
  {'first_word_index': 7,
   'phrase': 'a basket',
   'phrase_id': '84696',
   'phrase_type': ['other']},
  {'first_word_index': 12,
   'phrase': 'his head',
   'phrase_id': '84701',
   'phrase_type': ['bodyparts']},
  {'first_word_index': 16,
   'phrase': 'fruit',
   'phrase_id': '84706',
   'phrase_type': ['other']},
  {'first_word_index': 18,
   'phrase': 'vegetables',
   'phrase_id': '84696',
   'phrase_type': ['other']},
  {'first_word_index': 20,
   'phrase': 'one hand',
   'phrase_id': '84708',
   'phrase_type': ['bodyparts']},
  {'first_word_index': 25,
   'phrase': 'bags of grapes',
 